# Lab Assignment 4: Using APIs in Python
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will work with the public API provided by [genius.com](https://genius.com/), a website that calls itself "the world’s biggest collection of song lyrics and musical knowledge." You will need to read the API documentation carefully, acquire an access key, and use it without sharing it to pull data from this API into Python. You will also practice using a library made specifically to wrap around `requests` to make calling from the Genius API easier.

## Problem 0
Import the following libraries:

In [68]:
import numpy as np
import pandas as pd
import requests
import json
import os
import dotenv
import sys
sys.tracebacklimit = 0 # turn off the error tracebacks
os.chdir('C:\\Users\\Jacqueline\\OneDrive\\Documents\\MSDS\\CW\\lab4_6001')

## Problem 1
The Genius API documentation is here: https://docs.genius.com/#/getting-started-h1. Read through the documentation carefully. Although the Genius API is free and public, it still requires users an access key to use the API. In this case, Genius provides users with three codes: a client ID, a client ID secret, and a client access token. Use the documentation to find a way to obtain these codes for yourself. Write a paragraph that describes all of the steps you needed to take (but DO NOT list your access codes in this paragraph).

Some hints and cautions: 

1. Before you can use the API, you will need a regular, free user account with Genius. Sign-up here: https://genius.com/signup_or_login

2. Genius's API is built to support third-party app development, not data scientists. The language is entirely geared toward app development. Under "Authentication" there are instructions to third-party developers for guiding their own users in getting access keys. That's not relevant to getting access for yourself. It's not hard to get an access key, but the guidance here is not very clear. Be patient and read everything in the Authentication section carefully.

3. When you arrive at the page that allows you to register for API access keys, the language is still geared toward app development. You will be prompted to name your app and provide the URLs associated with the app. It doesn't much matter what you name your app, and I just used the Collab main page (https://collab.its.virginia.edu/portal?containerLogin=true) for the URLs.

4. When you get your codes, copy them in a text file. In problem 2 you will copy these codes over again to a `.env` file. [4 points]

#### Answer 1
I clicked "manage clients" and signed up with Google; took me straight to "Index API Client". Went back to the documentation to finish reading how to get all three codes. Clicked on "New API Client" and named my app "Jacqui App" and put all three urls as https://collab.its.virginia.edu/portal?containerLogin=true. I got the client ID and client key from doing just that. I saved both codes into my .env. Then after going back to the documentation page and authorizing by linking my Genius account, I scrolled down a little and found "Authorization: Bearer" and copied the access code there and put that as my access token. 

## Problem 2
Create a `.env` file for this project. Open it, copy your access codes into it, and save it. Then use Python code to load the environmental variables contained in the `.env` file, and create variables that contain each of the three codes. (You can print these variables to make sure it worked, but do not allow your access keys to display in your notebook). [4 points]

In [69]:
dotenv.load_dotenv()

True

In [70]:
os.getcwd()

'C:\\Users\\Jacqueline\\OneDrive\\Documents\\MSDS\\CW\\lab4_6001'

In [71]:
gid=os.getenv("gid")
gsecret=os.getenv("gsecret")
gaccess=os.getenv("gaccess")

## Problem 3
The root for all Genius APIs is https://api.genius.com. Find the endpoint for the Search API. (You will have to click the "Authorize with Genius" button in the upper-right corner if you haven't already done so). Use the `requests` library to issue a search for Bob Dylan. Genius's API is organized in a way that every individual artist has his or her own API endpoint. Display a portion of the JSON output that displays the API endpoint path for the data on Bob Dylan. 

Hint: to authenticate, specify your access token (not your client ID or client secret) as the `access_token` parameter. You will have to dig around the JSON output to find the artist ID, but it is listed under `primary_artist` several branches down the JSON tree. [4 points]

In [73]:
r = requests.get("https://httpbin.org/user-agent")
user_agent = json.loads(r.text)["user-agent"]
headers = {"User-Agent": user_agent,
          "From":"jdu5sq@virginia.edu"}
headers

{'User-Agent': 'python-requests/2.28.1', 'From': 'jdu5sq@virginia.edu'}

In [78]:
## Genius API
root = "https://api.genius.com"
endpt = "/search"
parameters = {"access_token": gaccess,
              "q": "Bob Dylan"}
r = requests.get(root+endpt, params = parameters, headers = headers)
r

<Response [200]>

In [79]:
dgenius = json.loads(r.text)["response"]
dgenius["hits"][0]

{'highlights': [],
 'index': 'song',
 'type': 'song',
 'result': {'annotation_count': 12,
  'api_path': '/songs/79424',
  'artist_names': 'Bob Dylan',
  'full_title': "Blowin' in the Wind by\xa0Bob\xa0Dylan",
  'header_image_thumbnail_url': 'https://images.genius.com/afd28edb1b2d6d5d556a7b01b108c716.300x300x1.jpg',
  'header_image_url': 'https://images.genius.com/afd28edb1b2d6d5d556a7b01b108c716.500x500x1.jpg',
  'id': 79424,
  'lyrics_owner_id': 73267,
  'lyrics_state': 'complete',
  'path': '/Bob-dylan-blowin-in-the-wind-lyrics',
  'pyongs_count': 48,
  'relationships_index_url': 'https://genius.com/Bob-dylan-blowin-in-the-wind-sample',
  'release_date_components': {'year': 1963, 'month': 5, 'day': 27},
  'release_date_for_display': 'May 27, 1963',
  'release_date_with_abbreviated_month_for_display': 'May. 27, 1963',
  'song_art_image_thumbnail_url': 'https://images.genius.com/afd28edb1b2d6d5d556a7b01b108c716.300x300x1.jpg',
  'song_art_image_url': 'https://images.genius.com/afd28edb

In [80]:
artist_id = dgenius["hits"][0]["result"]["primary_artist"]["id"]
artist_id

181

## Problem 4
Add `/songs` to the end of the the endpoint path you found in problem 3 and use this path to request the 20 most popular Bob Dylan songs. Organize these data in a `pandas` data frame. [4 points]

#### Answer 4

In [84]:
endpt = f"/artists/{artist_id}/songs"
parameters = {"access_token": gaccess}
r = requests.get(root+endpt, params=parameters, headers=headers)
r

<Response [200]>

In [110]:
songs = json.loads(r.text)["response"]["songs"]
songsdf = pd.json_normalize(songs)
songsdf

,annotation_count,api_path,artist_names,full_title,header_image_thumbnail_url,header_image_url,id,lyrics_owner_id,lyrics_state,path,...,primary_artist.api_path,primary_artist.header_image_url,primary_artist.id,primary_artist.image_url,primary_artist.is_meme_verified,primary_artist.is_verified,primary_artist.name,primary_artist.url,stats.pageviews,release_date_components
0,0,/songs/200681,Bob Dylan,"10,000 Men by Bob Dylan",https://images.genius.com/9a0da499b7383ddf2327...,https://images.genius.com/9a0da499b7383ddf2327...,200681,225680,complete,/Bob-dylan-10000-men-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
1,0,/songs/7105561,Bob Dylan & The Band,2 Dollars and 99 Cents by Bob Dylan & The Band,https://images.genius.com/84a940f13d290fc51835...,https://images.genius.com/84a940f13d290fc51835...,7105561,6190893,complete,/Bob-dylan-and-the-band-2-dollars-and-99-cents...,...,/artists/369744,https://images.genius.com/a703f2f12d927dd9f1ab...,369744,https://images.genius.com/a703f2f12d927dd9f1ab...,False,False,Bob Dylan & The Band,https://genius.com/artists/Bob-dylan-and-the-band,NaN,NaN
2,1,/songs/200682,Bob Dylan,2 X 2 by Bob Dylan,https://images.genius.com/9a0da499b7383ddf2327...,https://images.genius.com/9a0da499b7383ddf2327...,200682,225680,complete,/Bob-dylan-2-x-2-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
3,1,/songs/1686914,Bob Dylan,32-20 Blues by Bob Dylan,https://images.genius.com/dec36dae98920363912f...,https://images.genius.com/dec36dae98920363912f...,1686914,1549345,complete,/Bob-dylan-32-20-blues-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
4,15,/songs/105774,Bob Dylan,4th Time Around by Bob Dylan,https://images.genius.com/f96e173addad27a13065...,https://images.genius.com/f96e173addad27a13065...,105774,16,complete,/Bob-dylan-4th-time-around-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,38917.0,NaN
5,1,/songs/994912,Robyn Hitchcock,4th Time Around by Robyn Hitchcock,https://images.genius.com/4476c7b6ab3bd711b40f...,https://images.genius.com/4476c7b6ab3bd711b40f...,994912,1549345,complete,/Robyn-hitchcock-4th-time-around-lyrics,...,/artists/90989,https://images.genius.com/75e0332313e5e1a4c076...,90989,https://images.genius.com/75e0332313e5e1a4c076...,False,False,Robyn Hitchcock,https://genius.com/artists/Robyn-hitchcock,NaN,NaN
6,1,/songs/1493543,Traveling Wilburys,7 Deadly Sins by Traveling Wilburys,https://images.genius.com/eda586eb0d4993017d17...,https://images.genius.com/eda586eb0d4993017d17...,1493543,1549345,complete,/Traveling-wilburys-7-deadly-sins-lyrics,...,/artists/60612,https://images.genius.com/609e88d8b00dca0919f0...,60612,https://images.genius.com/c859d02ba3a25da3b1f9...,False,False,Traveling Wilburys,https://genius.com/artists/Traveling-wilburys,NaN,NaN
7,0,/songs/7103550,Bob Dylan & The Band,900 Miles from My Home by Bob Dylan & The Band,https://images.genius.com/84a940f13d290fc51835...,https://images.genius.com/84a940f13d290fc51835...,7103550,6190893,complete,/Bob-dylan-and-the-band-900-miles-from-my-home...,...,/artists/369744,https://images.genius.com/a703f2f12d927dd9f1ab...,369744,https://images.genius.com/a703f2f12d927dd9f1ab...,False,False,Bob Dylan & The Band,https://genius.com/artists/Bob-dylan-and-the-band,NaN,NaN
8,1,/songs/1994655,Bob Dylan,900 Miles from My Home by Bob Dylan,https://images.genius.com/571780cc8959528e8166...,https://images.genius.com/571780cc8959528e8166...,1994655,30682,complete,/Bob-dylan-900-mi

For this next part, I just wanted to see if I could get the songs where Bob Dylan was the primary artist.

In [107]:
songsd = json.loads(r.text)["response"]["songs"]
songsd_1 = [x for x in songsd if x["primary_artist"]["id"]==181]
songsd_1[1]

{'annotation_count': 1,
 'api_path': '/songs/200682',
 'artist_names': 'Bob Dylan',
 'full_title': '2 X 2 by\xa0Bob\xa0Dylan',
 'header_image_thumbnail_url': 'https://images.genius.com/9a0da499b7383ddf232734cfb860d33b.300x301x1.jpg',
 'header_image_url': 'https://images.genius.com/9a0da499b7383ddf232734cfb860d33b.500x501x1.jpg',
 'id': 200682,
 'lyrics_owner_id': 225680,
 'lyrics_state': 'complete',
 'path': '/Bob-dylan-2-x-2-lyrics',
 'pyongs_count': None,
 'relationships_index_url': 'https://genius.com/Bob-dylan-2-x-2-sample',
 'release_date_components': {'year': 1990, 'month': 9, 'day': 10},
 'release_date_for_display': 'September 10, 1990',
 'release_date_with_abbreviated_month_for_display': 'Sep. 10, 1990',
 'song_art_image_thumbnail_url': 'https://images.genius.com/9a0da499b7383ddf232734cfb860d33b.300x301x1.jpg',
 'song_art_image_url': 'https://images.genius.com/9a0da499b7383ddf232734cfb860d33b.500x501x1.jpg',
 'stats': {'unreviewed_annotations': 0, 'hot': False},
 'title': '2 X 

In [106]:
songsd2 = pd.json_normalize(songsd_1)
songsd2

,annotation_count,api_path,artist_names,full_title,header_image_thumbnail_url,header_image_url,id,lyrics_owner_id,lyrics_state,path,...,primary_artist.api_path,primary_artist.header_image_url,primary_artist.id,primary_artist.image_url,primary_artist.is_meme_verified,primary_artist.is_verified,primary_artist.name,primary_artist.url,stats.pageviews,release_date_components
0,0,/songs/200681,Bob Dylan,"10,000 Men by Bob Dylan",https://images.genius.com/9a0da499b7383ddf2327...,https://images.genius.com/9a0da499b7383ddf2327...,200681,225680,complete,/Bob-dylan-10000-men-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
1,1,/songs/200682,Bob Dylan,2 X 2 by Bob Dylan,https://images.genius.com/9a0da499b7383ddf2327...,https://images.genius.com/9a0da499b7383ddf2327...,200682,225680,complete,/Bob-dylan-2-x-2-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
2,1,/songs/1686914,Bob Dylan,32-20 Blues by Bob Dylan,https://images.genius.com/dec36dae98920363912f...,https://images.genius.com/dec36dae98920363912f...,1686914,1549345,complete,/Bob-dylan-32-20-blues-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
3,15,/songs/105774,Bob Dylan,4th Time Around by Bob Dylan,https://images.genius.com/f96e173addad27a13065...,https://images.genius.com/f96e173addad27a13065...,105774,16,complete,/Bob-dylan-4th-time-around-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,38917.0,NaN
4,1,/songs/1994655,Bob Dylan,900 Miles from My Home by Bob Dylan,https://images.genius.com/571780cc8959528e8166...,https://images.genius.com/571780cc8959528e8166...,1994655,30682,complete,/Bob-dylan-900-miles-from-my-home-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
5,3,/songs/118651,Bob Dylan (Ft. George Harrison),Abandoned Love by Bob Dylan (Ft. George Harrison),https://images.genius.com/3473ef0e3a751805b4da...,https://images.genius.com/3473ef0e3a751805b4da...,118651,188147,complete,/Bob-dylan-abandoned-love-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,8619.0,NaN
6,1,/songs/7236491,Bob Dylan,"Abraham, Martin, and John (Rehearsal) by Bob D...",https://images.genius.com/046f5a790beb6972f382...,https://images.genius.com/046f5a790beb6972f382...,7236491,7486996,complete,/Bob-dylan-abraham-martin-and-john-rehearsal-l...,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,NaN,NaN
7,7,/songs/105773,Bob Dylan,Absolutely Sweet Marie by Bob Dylan,https://images.genius.com/f96e173addad27a13065...,https://images.genius.com/f96e173addad27a13065...,105773,16,complete,/Bob-dylan-absolutely-sweet-marie-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://images.genius.com/571780cc8959528e8166...,False,False,Bob Dylan,https://genius.com/artists/Bob-dylan,18817.0,NaN
8,0,/songs/6223061,Bob Dylan,Absolutely Sweet Marie (Take 1) by Bob Dylan,https://images.genius.com/2b3b36c03d916ef7d281...,https://images.genius.com/2b3b36c03d916ef7d281...,6223061,2749279,incomplete,/Bob-dylan-absolutely-sweet-marie-take-1-lyrics,...,/artists/181,https://images.genius.com/e87fb11dd7f33cc7fd1a...,181,https://

## Problem 5
Install and import the `lyricsgenius` library in Python, which is a wrapper around `requests` that works specifically with the Genius API. . Follow the guide on the GitHub repository for this library (https://github.com/johnwmillr/LyricsGenius) for instructions on using the library. Use the `lyricsgenius` library to download and display the lyrics to "Tangled Up in Blue" by Bob Dylan. [4 points]

#### Answer 5

In [112]:
# !pip install lyricsgenius

In [113]:
import lyricsgenius
genius = lyricsgenius.Genius(gaccess)

In [114]:
song = genius.search_song("Tangled Up in Blue", "Bob Dylan")
print(song.lyrics)

Searching for "Tangled Up in Blue" by Bob Dylan...
Done.
91 ContributorsTangled Up in Blue Lyrics[Verse 1]
Early one morning the sun was shining
I was laying in bed
Wondering if she'd changed at all
If her hair was still red
Her folks they said our lives together
Sure was going to be rough
They never did like Mama's homemade dress
Papa's bankbook wasn't big enough
And I was standing on the side of the road
Rain falling on my shoes
Heading out for the East Coast
Lord knows I've paid some dues
Getting through
Tangled up in blue

[Verse 2]
She was married when we first met
Soon to be divorced
I helped her out of a jam, I guess
But I used a little too much force
We drove that car as far as we could
Abandoned it out west
Split up on a dark sad night
Both agreeing it was best
She turned around to look at me
As I was walking away
I heard her say over my shoulder
"We'll meet again someday
On the avenue"
Tangled up in blue
See Bob Dylan LiveGet tickets as low as $124You might also like[Verse 3]